<a href="https://colab.research.google.com/github/phamnguyenlongvu/LLMs/blob/main/Fine_tuning_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install peft
!pip install torch
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.4 MB/s eta 0:00:00


In [ ]:
!pip show bitsandbytes

Name: bitsandbytes
Version: 0.45.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch, typing_extensions
Required-by: 


In [ ]:
import locale
import pprint

# def getpreferredecoding(do_setlocate=True):
#     return "UTF_8"

# locale.getpreferredencoding = getpreferredecoding

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
import transformers
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
model_id = "tiiuae/Falcon3-1B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = bnb_config,
    device_map="auto",
    trust_remote_code=True
)

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training

pretrained_model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(pretrained_model)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable params: {trainable_params} || all params: {all_param} || trainable%: {100* trainable_params/all_param}")

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    # target_modules=["query_key_value"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

Trainable params: 2064384 || all params: 1105242112 || trainable%: 0.18678115659784053


In [ ]:
from datasets import load_dataset

data = load_dataset("celikmus/mayo_clinic_symptoms_and_diseases_v1", split="train")

README.md:   0%|          | 0.00/424 [00:00<?, ?B/s]

(…)-00000-of-00001-8cc6b347d333d935.parquet:   0%|          | 0.00/626k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

train_dataset = data.map(
    lambda x: {
        "input_text": f"symptoms: {x['text']}; most likely explanation: {x['label']}"
    }
)

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

In [ ]:
train_dataset['input_text'][1]

'symptoms: The signs and symptoms of benign paroxysmal positional vertigo (BPPV) may include: \nDizziness\nA sense that you or your surroundings are spinning or moving (vertigo)\nA loss of balance or unsteadiness\nNausea\nVomiting\n The signs and symptoms of BPPV can come and go and commonly last less than one minute. Episodes of BPPV can disappear for some time and then recur. BPPV BPPV Activities that bring about the signs and symptoms of BPPV can vary from person to person, but are almost always brought on by a change in head position. Some people also feel out of balance when standing or walking. BPPV Abnormal rhythmic eye movements usually accompany the symptoms of benign paroxysmal positional vertigo.; most likely explanation: vertigo'

In [ ]:
train_encodings = tokenizer(
    train_dataset["input_text"],
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors="pt"
)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()

        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = TextDataset(train_encodings)


In [ ]:
request_text = """
In crowded places, I feel cold in the tips of my fingers, I sweat with dizziness.
What is happening?
"""

In [ ]:
encoding = tokenizer(request_text, return_tensors="pt").to("cuda:0")
print(encoding)
pretrained_model_output = pretrained_model.generate(
    input_ids=encoding.input_ids,
    attention_mask=encoding.attention_mask,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.25,
    eos_token_id=tokenizer.eos_token_id,
)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


{'input_ids': tensor([[   12,  2705, 22754,  6208,  2035,  2332,  3365,  6686,  2303,  2278,
          7448,  2306,  2584, 12857,  2035,  2332, 14899,  2368, 52449,  2037,
            12,  4042,  2340,  9571,  2054,    12]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]], device='cuda:0')}


In [ ]:
pprint.pprint(tokenizer.decode(pretrained_model_output[0], skip_special_tokens=True))

('\n'
 'In crowded places, I feel cold in the tips of my fingers, I sweat with '
 'dizziness.\n'
 'What is happening?\n'
 '\n'
 "This is a classic symptom of a condition called Raynaud's disease. It's a "
 "disorder that affects blood flow to your fingers and toes. When you're "
 'exposed to cold or stress, your body tries to warm up the blood vessels to '
 'the fingers and toes, which can cause them to narrow. This reduces blood '
 'flow to the fingers and toes, making them feel cold and causing them to turn '
 "white or blue (a condition called Raynaud's phenomenon).\n"
 '\n'
 'People with')


In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=10,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=40,
        learning_rate=2.5e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type="cosine",
        report_to="none",
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False

In [ ]:
trainer.train()

<ipython-input-25-e8697bcf73ea>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.255600
2,2.275900
3,2.291400
4,2.228200
5,2.307700
6,2.127400
7,2.047800
8,2.185600
9,2.127300
10,2.155600


TrainOutput(global_step=40, training_loss=2.004554036259651, metrics={'train_runtime': 234.9137, 'train_samples_per_second': 5.449, 'train_steps_per_second': 0.17, 'total_flos': 2693832376320000.0, 'train_loss': 2.004554036259651, 'epoch': 1.1804511278195489})